In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [6]:
def preprocess_file(file_path: str) -> pd.DataFrame:
    curr = pd.read_csv(file_path, delimiter=';')
    
    if curr.isna().sum().sum():
        print(f"Null values found in: {file_path}")
        return None
    
    curr["time"] = pd.to_datetime(curr["time"])
    curr.sort_values("time", inplace=True)
    time_deltas = curr["time"].diff().dropna()
    
    if not (time_deltas == pd.Timedelta(minutes=5)).all():
        print(f"The samples were not in the 5 min interval here: {file_path}")
        return None
    
    return curr

In [7]:
def preprocess_data(dir_path: str) -> pd.DataFrame:
    all_dfs = []
    files = [f for f in os.listdir(dir_path) if f.endswith(".csv")]
    
    for index, filename in enumerate(sorted(files)):
        full_path = os.path.join(dir_path, filename)
        curr = preprocess_file(full_path)
        if curr is not None:
            curr["user_id"] = index + 1
            all_dfs.append(curr)
            
    return pd.concat(all_dfs, ignore_index=True)

In [8]:
db = preprocess_data("data")
db.head()

,time,glucose,calories,heart_rate,steps,basal_rate,bolus_volume_delivered,carb_input,user_id
0,2018-06-13 18:40:00,332.0,6.3595,82.322835,34.0,0.091667,0.0,0.0,1
1,2018-06-13 18:45:00,326.0,7.7280,83.740157,0.0,0.091667,0.0,0.0,1
2,2018-06-13 18:50:00,330.0,4.7495,80.525180,0.0,0.091667,0.0,0.0,1
3,2018-06-13 18:55:00,324.0,6.3595,89.129032,20.0,0.091667,0.0,0.0,1
4,2018-06-13 19:00:00,306.0,5.1520,92.495652,0.0,0.075000,0.0,0.0,1


In [9]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309392 entries, 0 to 309391
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   time                    309392 non-null  datetime64[ns]
 1   glucose                 309392 non-null  float64       
 2   calories                309392 non-null  float64       
 3   heart_rate              309392 non-null  float64       
 4   steps                   309392 non-null  float64       
 5   basal_rate              309392 non-null  float64       
 6   bolus_volume_delivered  309392 non-null  float64       
 7   carb_input              309392 non-null  float64       
 8   user_id                 309392 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 21.2 MB


In [10]:
db.describe()

,time,glucose,calories,heart_rate,steps,basal_rate,bolus_volume_delivered,carb_input,user_id
count,309392,309392.000000,309392.000000,309392.000000,309392.000000,309392.000000,309392.000000,309392.000000,309392.000000
mean,2020-09-24 13:06:07.680159232,141.425051,8.813568,76.990001,30.825005,0.041324,0.066058,0.052718,20.813066
min,2018-06-13 18:40:00,40.000000,0.000000,32.407773,0.000000,0.000000,-3.000000,0.000000,1.000000
25%,2020-01-30 11:00:00,99.666667,5.846100,64.930233,0.000000,0.000000,0.000000,0.000000,22.000000
50%,2020-10-31 16:52:30,132.000000,6.278100,75.418726,0.000000,0.056000,0.000000,0.000000,24.000000
75%,2021-07-27 06:31:15,173.000000,9.193060,85.612685,11.000000,0.066000,0.000000,0.000000,24.000000
max,2022-05-18 12:15:00,444.000000,106.350000,195.615385,842.000000,0.250000,19.800000,130.000000,25.000000
std,NaN,57.085587,6.930449,15.546699,84.981109,0.036106,0.755075,1.505433,6.283139


In [11]:
db.to_csv("data/db.csv")

In [13]:
synthetic = pd.read_csv("synthetic/synt.csv")
labels = synthetic.iloc[:, 0]
synthetic = synthetic.iloc[:, 1:]
synthetic.columns = [f"t_{i}" for i in range(1, synthetic.shape[1] + 1)]
synthetic = pd.concat([labels.rename("type"), synthetic], axis=1)
#synthetic.to_csv("synthetic/prepr_synt.csv")

In [14]:
def make_smaller_by_type(df: pd.DataFrame, typ: int, num: int) -> pd.DataFrame:
    smaller_df = df[df["type"] == typ]
    smaller_df = smaller_df.iloc[:num,]
    return smaller_df